In [176]:
library(glmnet)
library(doMC)
library(survival)
library(data.table)
library(mltools)
library(CoxBoost)
library(randomForestSRC)
library(CoxHD)
source('run_prognosis.R')

In [20]:
df_all <-read.table("df_prognosis.tsv",sep = '\t' , header = T) 

###    2094 rows 166 columns

df_all <-na.omit(df_all) # delete rows with na (161)
df_all <- df_all[df_all$os>0,] # delete when os is negative (2)

####

#Convert predicted_component to one hot encoder
df_all$new_eln<-factor(df_all$new_eln, levels = c("adverse","intermediate","favorable"), labels = 0:2, ordered = TRUE)  # convert categorical new_eln to numerical (0,1,2)
name <-rownames(df_all)
df_all$predicted_component <- as.factor(df_all$predicted_component)
df_final <- as.data.frame(one_hot(as.data.table(df_all),cols="predicted_component"))
rownames(df_final) <- name

#write.table(df_final,"df_prognosis_features_ready.tsv",sep='\t',quote=F)
####

###  1931 rows 180 columns

In [21]:
#dim(df_all)
#dim(df_final)
df_final <- read.table("df_prognosis_features_ready.tsv",sep = '\t' , header = T)
x <- data.matrix(df_final[,1:84])
y <- data.matrix(df_final[,c("os","os_status")])
predictors <- c(predictorGLM)
#predictors <- c(rep(list(predictorGLM),10),rep(list(predictorRF),10),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxMod",10),rep("RFS",10),"CoxBoost","RFX")
l_alpha <-seq(0.1,1,0.1)
l_ntree <- seq(100,1000,100)
launch_prognosis(x,y,predictors,str_predictors,l_alpha,l_ntree)

[1] "start CV"


CoxMod_0.1
0.6265973
0.6511334
0.6376802
0.6470114
0.6554482
0.6808461
0.6326469
0.6370511
0.6442483
0.6153510


In [185]:
x <- data.matrix(df_final[,1:84])
y <- data.matrix(df_final[,c("os","os_status")])
predictors <- c(rep(list(predictorGLM),3),predictorRF,predictorRF,predictorBoost)
#predictors <- c(predictorRFX)
#predictors <- c(rep(list(predictorGLM),10),rep(list(predictorRF),10),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxMod",10),rep("RFS",10),"CoxBoost","RFX")
str_predictors <- c(rep("CoxGLM",3),rep("predictorRF",4),"predictorBoost")
l_alpha <-seq(0.05,1,0.05)
#l_ntree <- seq(100,1500,100)
l_ntree <-c(10,20)
(launch_prognosis(x=x,y=y,predictors=predictors,str_predictors=str_predictors,l_alpha=l_alpha,l_ntree=l_ntree,mc.cores=1,nodesize=c(10,20),nrepeats=1,nfolds=2))

[1] "start CV"
..[1] "start CV"
..[1] "start CV"
..[1] "start CV"
.[1] 10
.[1] 10
[1] "start CV"
.[1] 20
.[1] 20
[1] "start CV"
.[1] 10
.[1] 10
[1] "start CV"
.[1] 20
.[1] 20
[1] "start CV"
..

CoxGLM_0.05,CoxGLM_0.1,CoxGLM_0.15,predictorRF_10_10,predictorRF_10_20,predictorRF_20_10,predictorRF_20_20,predictorBoost_
0.6478436,0.6479093,0.6471204,0.6398329,0.6387570,0.6347908,0.6448320,0.6471647
0.6498326,0.6485409,0.6485937,0.6363847,0.6407016,0.6454629,0.6515815,0.6474822


In [218]:
#colnames(df_final)
all_features <- 1:177
eln_clin<-c(1,169:175)
eln_clin_demo<-c(1,169:177)
eln_clin_demo_cyto <-c(1,85:153,169:177)
eln_clin_demo_gen <-c(1:84,169:177)
eln_clin_demo_cyto_gen <-c(1:153,169:177)
eln_clin_demo_comp <-c(1,154:177)
eln_cyto_gen<-c(1:153)
eln_cyto_gen_comp <-c(1:168)
eln_cyto_comp <-c(1,85:168)
eln_gen_comp <-c(1:84,154:168)

clin_demo<-c(169:177)
clin_demo_cyto <-c(85:153,169:177)
clin_demo_gen <-c(2:84,169:177)
clin_demo_cyto_gen <-c(2:153,169:177)
clin_demo_comp <-c(154:177)
cyto_gen<-c(2:153)
cyto_gen_comp <-c(2:168)
cyto_comp <-c(85:168)
gen_comp <-c(2:84,154:168)
clin_demo_cyto_gen_comp<-c(2:177)
gen<-c(2:84)
cyto<-c(85:153)
comp<-c(154:168)
a <-list(all_features=all_features,eln_clin=eln_clin,eln_clin_demo=eln_clin_demo,
    eln_clin_demo_cyto=eln_clin_demo_cyto,eln_clin_demo_gen=eln_clin_demo_gen,
    eln_clin_demo_cyto_gen=eln_clin_demo_cyto_gen,eln_clin_demo_comp=eln_clin_demo_comp,
    eln_cyto_gen=eln_cyto_gen,eln_cyto_gen_comp=eln_cyto_gen_comp,eln_cyto_comp=eln_cyto_comp,
    eln_gen_comp=eln_gen_comp,clin_demo=clin_demo,clin_demo_cyto=clin_demo_cyto,clin_demo_gen=clin_demo_gen,
    clin_demo_cyto_gen=clin_demo_cyto_gen,clin_demo_comp=clin_demo_comp,cyto_gen=cyto_gen,cyto_gen_comp=cyto_gen_comp,
    cyto_comp=cyto_comp,gen_comp=gen_comp,clin_demo_cyto_gen_comp=clin_demo_cyto_gen_comp,gen=gen,cyto=cyto,comp=comp)

In [240]:
prognosis_features<- list(all_features=all_features,eln_clin=eln_clin,eln_clin_demo=eln_clin_demo,
    eln_clin_demo_cyto=eln_clin_demo_cyto,eln_clin_demo_gen=eln_clin_demo_gen,
    eln_clin_demo_cyto_gen=eln_clin_demo_cyto_gen,eln_clin_demo_comp=eln_clin_demo_comp,
    eln_cyto_gen=eln_cyto_gen,eln_cyto_gen_comp=eln_cyto_gen_comp,eln_cyto_comp=eln_cyto_comp,
    eln_gen_comp=eln_gen_comp,clin_demo=clin_demo,clin_demo_cyto=clin_demo_cyto,clin_demo_gen=clin_demo_gen,
    clin_demo_cyto_gen=clin_demo_cyto_gen,clin_demo_comp=clin_demo_comp,cyto_gen=cyto_gen,cyto_gen_comp=cyto_gen_comp,
    cyto_comp=cyto_comp,gen_comp=gen_comp,clin_demo_cyto_gen_comp=clin_demo_cyto_gen_comp,gen=gen,cyto=cyto,comp=comp)
###--------------------------------------------------
y <- data.matrix(df_final[,c("os","os_status")])

predictors <- c(rep(list(predictorGLM),20),rep(list(predictorRF),15),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxGLM",20),rep("RFS",105),"CoxBoost","RFX")
l_alpha <-seq(0.05,1,0.05)
l_ntree <- seq(100,1500,100)
mc.cores <- 8
nodesize <- c(5,10,15,20,30,40,50)
for (i in 1:length(prognosis_features)){
    x <- data.matrix(df_final[,prognosis_features[[i]]])
    print(paste(names(prognosis_features)[i],".tsv",sep=""))
    print("DONE")
    }

[1] "all_features.tsv"
[1] "DONE"
[1] "eln_clin.tsv"
[1] "DONE"
[1] "eln_clin_demo.tsv"
[1] "DONE"
[1] "eln_clin_demo_cyto.tsv"
[1] "DONE"
[1] "eln_clin_demo_gen.tsv"
[1] "DONE"
[1] "eln_clin_demo_cyto_gen.tsv"
[1] "DONE"
[1] "eln_clin_demo_comp.tsv"
[1] "DONE"
[1] "eln_cyto_gen.tsv"
[1] "DONE"
[1] "eln_cyto_gen_comp.tsv"
[1] "DONE"
[1] "eln_cyto_comp.tsv"
[1] "DONE"
[1] "eln_gen_comp.tsv"
[1] "DONE"
[1] "clin_demo.tsv"
[1] "DONE"
[1] "clin_demo_cyto.tsv"
[1] "DONE"
[1] "clin_demo_gen.tsv"
[1] "DONE"
[1] "clin_demo_cyto_gen.tsv"
[1] "DONE"
[1] "clin_demo_comp.tsv"
[1] "DONE"
[1] "cyto_gen.tsv"
[1] "DONE"
[1] "cyto_gen_comp.tsv"
[1] "DONE"
[1] "cyto_comp.tsv"
[1] "DONE"
[1] "gen_comp.tsv"
[1] "DONE"
[1] "clin_demo_cyto_gen_comp.tsv"
[1] "DONE"
[1] "gen.tsv"
[1] "DONE"
[1] "cyto.tsv"
[1] "DONE"
[1] "comp.tsv"
[1] "DONE"


In [181]:
df_final <- read.table("df_prognosis_features_ready.tsv",sep = '\t' , header = T)
x <- data.matrix(df_final[,1:84])
y <- data.matrix(df_final[,c("os","os_status")])
predictors <- c(rep(list(predictorGLM),20),rep(list(predictorRF),105),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxGLM",20),rep("RFS",105),"CoxBoost","RFX")
l_alpha <-seq(0.05,1,0.05)
l_ntree <- seq(100,1500,100)
#write.table(launch_prognosis(x,y,predictors,str_predictors,l_alpha,l_ntree),"tests.tsv",quote=F,sep='\t')
launch_prognosis(x=x,y=y,predictors=predictors,str_predictors=str_predictors,l_alpha=l_alpha,l_ntree=l_ntree,mc.cores=8,nodesize=c(5,10,15,20,30,40,50))

In [177]:
mean(launch_prognosis(x=x,y=y,predictors=c(predictorRFX),str_predictors=str_predictors,l_alpha=l_alpha,l_ntree=l_ntree,mc.cores=1,nodesize=c(10,20),nrepeats=2,nfolds=5))

[1] "start CV"
..........

[1] 0.6474806

In [104]:
mean(launch_prognosis(x,y,c(predictorRF),str_predictors,l_alpha,l_ntree,mc.cores=1))

[1] "start CV"
.........................

[1] 0.6480227

In [164]:
predictorRFX


function (designTrain, designTest, responseTrain, max.iter = 500, 
    tol = 1e-04) 
{
    set.seed(17)
    cvfit = CoxRFX(data.frame(designTrain), Surv(time = responseTrain[, 
        1], event = responseTrain[, 2]), max.iter = max.iter, 
        tol = tol)
    cvfit$Z <- NULL
    risk.predict <- predict(cvfit, data.frame(designTest))
    return(risk.predict)
}

In [77]:
help(CoxRFX)

CoxRFX {CoxHD},R Documentation
Z,The data matrix of random effects (n x p)
surv,The survival object (n x 2)
groups,"Optional groups as a factor (p) with l levels. Default = rep(1, n)"
which.mu,Indicator which of the groups should have an offset. Default = unique(groups)
tol,The tolerance beyond which to stop
max.iter,The maximal number of iterations
sigma0,The variance of a si-chisq hyperprior on the variances.
nu,"The df of the variance hyperprior. Default = 0, that is no hyperprior."
penalize.mu,"Wether to define an N(0,tau) hyperprior on the group means."
sigma.hat,"Which estimator to use for the variances. Default df, other possibilities include MLE, REML and BLUP, see details."
